In [12]:
import pandas as pd
from databaker.framework import *
from databakerUtils.writers import v4Writer
import requests

inputFile = 'localauthoritiesregistrations201517.xlsx'
outputFile = 'v4_drug-related-deaths.csv'

tabsWeWant = ['Poisoning persons', 'Poisoning males', 'Poisoning females', 
              'Misuse persons', 'Misuse males', 'Misuse females']

tabs = loadxlstabs(inputFile, tabsWeWant)

conversionsegments = []

for tab in tabs:
    
    assert tab.excel_ref('A3').value == 'Area Code'
    
    key = tab.excel_ref('A').filter(contains_string('Age-standardised mortality')).expand(DOWN)
    
    geogCodes = tab.excel_ref('A6').expand(DOWN).is_not_blank().is_not_whitespace()
    geogCodes -= key
    geogLabels = tab.excel_ref('B6:D6').expand(DOWN).is_not_blank().is_not_whitespace()
    
    time = tab.excel_ref('E3').expand(RIGHT).is_not_blank().is_not_whitespace()
    
    mortality = tab.excel_ref('E4').expand(RIGHT).is_not_blank().is_not_whitespace()
    
    sex = tab.name.split()[1]
    
    typeOfDeath = tab.name.split()[0]
    
    obs = geogCodes.waffle(mortality)
    
    dimensions = [
                HDim(time, TIME, CLOSEST, LEFT),
                HDim(geogCodes, GEOG, DIRECTLY, LEFT),
                HDim(geogLabels, 'geogLabels', DIRECTLY, LEFT),
                HDim(mortality, 'mortality', DIRECTLY, ABOVE),
                HDimConst('sex', sex),
                HDimConst('typeOfDeath', typeOfDeath)
                ]

    conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
    conversionsegments.append(conversionsegment)
    
    print(tab.name,'- databaked')

data = pd.concat(conversionsegments)   
data = data.reset_index(drop = True)

#combing CV's into correct place
#concat it twice because the CV's are the same for the deaths and the rates
lowerCV = data[data['mortality'] == 'Lower Confidence Limit']
lowerCV = pd.concat([lowerCV, lowerCV])
lowerCV = lowerCV.sort_index()
upperCV = data[data['mortality'] == 'Upper Confidence Limit']
upperCV = pd.concat([upperCV, upperCV])
upperCV = upperCV.sort_index()

#dropping CV's from mortality column
data = data[data['mortality'] != 'Lower Confidence Limit']
data = data[data['mortality'] != 'Upper Confidence Limit']

df = v4Writer(outputFile, data, asFrame=True) 
df = df.reset_index(drop = True)  
lowerCV = lowerCV.reset_index(drop = True)
upperCV = upperCV.reset_index(drop = True)

df['Lower Confidence Limit'] = lowerCV['OBS']
df['Upper Confidence Limit'] = upperCV['OBS']

'''Functions'''

def sexLabels(value):
    lookup = {
            'persons':'All',
            'males':'Male',
            'females':'Female'
            }
    return lookup[value]

url = 'https://api.beta.ons.gov.uk/v1/code-lists/admin-geography/editions/one-off/codes'
r = requests.get(url)
wholeDict = r.json()
adminDict = {}
for item in wholeDict['items']:
    adminDict.update({item['id']:item['label']})
    
def adminLabels(value):
    return adminDict[value]

'''Post Processing'''

df['Time_codelist'] = df['Time']

df['Geography'] = df['Geography_codelist'].apply(adminLabels)
df = df.drop(['geogLabels', 'geogLabels_codelist'], axis = 1)

df['mortality_codelist'] = df['mortality'].apply(lambda x:x.lower())

df['sex'] = df['sex'].apply(sexLabels)
df['sex_codelist'] = df['sex'].apply(lambda x:x.lower())

df['typeOfDeath_codelist'] = df['typeOfDeath'].apply(lambda x:x.lower())

df = df[['V4_0', 'Data_Marking', 'Lower Confidence Limit', 'Upper Confidence Limit', 
         'Time_codelist', 'Time', 'Geography_codelist', 'Geography', 
         'mortality_codelist', 'mortality', 'sex_codelist', 'sex', 
         'typeOfDeath_codelist', 'typeOfDeath', ]]

renameCols = {
            'V4_0':'V4_3',
            'Time_codelist':'two-year-intervals',
            'Time':'time',
            'Geography_codelist':'admin-geography',
            'Geography':'geography',
            'mortality_codelist':'drug-deaths-mortality',
            'sex_codelist':'ashe-sex',
            'typeOfDeath_codelist':'drug-deaths-type-of-death',
            'typeOfDeath':'typeofdeath'
            }

df = df.rename(columns = renameCols)
df = df.reset_index(drop = True)

'''Removing data for codes starting E11 or E13 - they have no data but some datamarkings..'''

codesWithNoData = []
for code in df['admin-geography'].unique():
    if code.startswith('E11') or code.startswith('E13'):
        codesWithNoData.append(code)

codeList = list(df['admin-geography'])
indexOfCodesWithNoData = []
for index, code in enumerate(codeList):
    if code in codesWithNoData:
        indexOfCodesWithNoData.append(index)
    
df = df.drop(indexOfCodesWithNoData)

df.to_csv(outputFile, index = False)


Loading localauthoritiesregistrations201517.xlsx which has size 1250896 bytes
Table names: ['Poisoning persons', 'Poisoning males', 'Poisoning females', 'Misuse persons', 'Misuse males', 'Misuse females']
TIMEUNIT=''
Poisoning persons - databaked
TIMEUNIT=''
Poisoning males - databaked
TIMEUNIT=''
Poisoning females - databaked
TIMEUNIT=''
Misuse persons - databaked
TIMEUNIT=''
Misuse males - databaked
TIMEUNIT=''
Misuse females - databaked
Extracting data structured as v4.
